# Clustering Crypto

In [22]:
!pip install hvplot

In [23]:
!pip install plotly

In [24]:
!pip install sklearn

In [27]:
# Initial imports
import pandas as pd
# import hvplot.pandas
from pathlib import Path
# import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Deliverable 1: Preprocessing the Data for PCA

In [10]:
file_path = "./crypto_data.csv" 
df = pd.read_csv(file_path, index_col=0)
df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [11]:
# Keep all the cryptocurrencies that are being traded.
df1 = df.loc[df['IsTrading'] == True]
df1.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [12]:
# Keep all the cryptocurrencies that have a working algorithm.
df0 = df1.sort_values(by='Algorithm', ascending=False)
df0.tail()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
AQUA,Aquachain,Argon2,True,PoW,0.000000e+00,42000000
OPES,Opes,Argon2,True,PoW,NaN,52000000
BOAT,Doubloon,536,True,PoW/PoS,NaN,500000000
ESP,Espers,536,True,PoW/PoS,2.280188e+10,50000000000
HODL,HOdlcoin,1GB AES Pattern Search,True,PoW,1.144895e+07,81962100


In [13]:
df2 = df1.dropna(axis=0, subset=['Algorithm'])
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1144 entries, 42 to XBC
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CoinName         1144 non-null   object 
 1   Algorithm        1144 non-null   object 
 2   IsTrading        1144 non-null   bool   
 3   ProofType        1144 non-null   object 
 4   TotalCoinsMined  685 non-null    float64
 5   TotalCoinSupply  1144 non-null   object 
dtypes: bool(1), float64(1), object(4)
memory usage: 54.7+ KB


In [14]:
# Remove the "IsTrading" column. 
df3 = df2.drop(['IsTrading'], axis=1)
df3.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [15]:
# Remove rows that have at least 1 null value.
df4 = df3.dropna()
df4.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [16]:
# Keep the rows where coins are mined.
df5 = df4.loc[df4['TotalCoinsMined'] > 0]
df5.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [17]:
# Create a new DataFrame that holds only the cryptocurrencies names.
cc_names_df = df5[["CoinName"]]
cc_names_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [18]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = df5.drop(['CoinName'], axis=1)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [19]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled[0:5])

[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

### Deliverable 2: Reducing Data Dimensions Using PCA

In [29]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
pca

PCA(n_components=3)

In [30]:
# Create a DataFrame with the three principal components.
X_pca = pca.fit_transform(X_scaled)
X_pca

array([[-0.32308322,  1.07314993, -0.56826607],
       [-0.30646823,  1.0735306 , -0.56870274],
       [ 2.29154866,  1.78169415, -0.66611248],
       ...,
       [ 0.32103077, -2.34116081,  0.4006315 ],
       [-0.24803459, -1.8889826 ,  0.37082468],
       [-0.28594619,  0.89715191, -0.25537419]])

In [31]:
index_values = (X.index.tolist())

In [32]:
pcs_df = pd.DataFrame(data = X_pca, columns=["PC 1", "PC 2", "PC 3"], index = index_values)
pcs_df.head()

,PC 1,PC 2,PC 3
42,-0.323083,1.073150,-0.568266
404,-0.306468,1.073531,-0.568703
1337,2.291549,1.781694,-0.666112
BTC,-0.149701,-1.321289,0.202538
ETH,-0.140991,-2.063303,0.386233


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [33]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

In [34]:
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

C:\Users\19132\anaconda3\envs\PythonData\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  f"KMeans is known to have a memory leak on Windows "


In [38]:
# Plot the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
# df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)
# hvplot not working

Running K-Means with `k=4`

In [37]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
print(predictions)
pcs_df["Class"] = model.labels_

[0 0 0 2 2 2 0 2 2 2 0 2 0 0 2 0 2 2 0 0 2 2 2 2 2 0 2 2 2 0 2 0 2 2 0 0 2
 2 2 2 2 2 0 0 2 2 2 2 2 0 0 2 0 2 2 2 2 0 2 2 0 2 0 0 0 2 2 2 0 0 0 0 0 2
 2 2 0 0 2 0 2 0 0 2 2 2 2 0 0 2 0 2 2 0 0 2 0 0 2 2 0 0 2 0 0 2 0 2 0 2 0
 2 0 0 2 2 0 2 2 2 0 2 2 2 2 2 0 0 2 2 2 0 2 0 2 2 0 2 0 2 0 0 2 2 0 2 2 0
 0 2 0 2 0 0 0 2 2 2 2 0 0 0 0 0 2 2 0 0 0 0 0 2 0 0 0 0 0 2 0 2 0 0 2 0 2
 0 0 2 0 2 0 2 0 2 0 0 0 0 2 0 0 0 0 0 2 2 0 0 2 2 0 0 0 0 0 2 0 0 0 0 0 0
 0 0 2 0 0 0 0 0 0 2 2 2 0 0 0 0 2 0 2 0 0 2 0 2 2 0 2 2 0 2 0 0 0 2 0 0 2
 0 0 0 0 0 0 0 2 0 2 0 0 0 0 2 0 2 0 2 2 2 2 0 2 0 0 2 0 2 2 2 0 2 0 2 2 2
 0 2 0 2 0 0 3 2 0 2 2 2 2 2 0 0 2 0 0 0 2 0 2 0 2 0 2 0 0 0 0 2 0 0 2 0 0
 0 2 2 2 2 0 0 0 0 2 0 2 2 2 0 0 2 2 0 0 2 0 2 2 2 0 2 2 0 0 0 2 2 2 0 0 0
 2 2 0 2 2 2 2 0 3 3 2 2 2 0 3 0 0 0 0 2 2 2 2 0 0 0 2 0 2 0 0 0 0 2 0 0 2
 0 0 2 2 0 2 0 2 2 2 2 0 0 2 0 2 0 0 0 0 0 0 2 2 2 0 0 0 0 0 0 2 0 2 2 2 2
 0 0 0 0 2 0 0 2 0 0 2 3 2 0 2 2 0 0 2 0 2 2 0 2 2 0 2 0 2 0 0 2 0 0 0 0 0
 2 2 2 0 0 0 2 0 2 0 2 0 

In [39]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = crypto_df.join(pcs_df, how='inner')
clustered_df.head()

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df = clustered_df.join(cc_names_df, how='inner')
clustered_df.head()

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# Accomplished with inner join

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,Class,CoinName
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.323083,1.073150,-0.568266,0,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.306468,1.073531,-0.568703,0,404Coin
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.291549,1.781694,-0.666112,0,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.149701,-1.321289,0.202538,2,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0,-0.140991,-2.063303,0.386233,2,Ethereum
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.155686,-1.086780,-0.006764,2,Litecoin
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.391185,1.119911,-0.527327,0,Dash
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.120725,-2.396485,0.369522,2,Monero
ETC,Ethash,PoW,1.133597e+08,210000000,-0.139438,-2.063379,0.386210,2,Ethereum Classic
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.248034,-1.888983,0.370825,2,ZCash


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [40]:
# Creating a 3D-Scatter with the PCA data and the clusters
# Plotly not working
fig = px.scatter_3d(
    clustered_df,
    x="PC 3",
    y="PC 2",
    z="PC 1",
    hover_name="CoinName",
    hover_data=["Algorithm"],
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


NameError: name 'px' is not defined

In [41]:
# Create a table with tradable cryptocurrencies.
# hvplot not working
clustered_df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "class"], width=800)

AttributeError: 'DataFrame' object has no attribute 'hvplot'

In [42]:
# Print the total number of tradable cryptocurrencies.
count= clustered_df['CoinName'].count()
print(f'There are {count} tradable cryptocurrencies')

There are 532 tradable cryptocurrencies


In [44]:
supply_mined_df = clustered_df[['TotalCoinSupply', 'TotalCoinsMined']]
supply_mined_df

,TotalCoinSupply,TotalCoinsMined
42,42,4.199995e+01
404,532000000,1.055185e+09
1337,314159265359,2.927942e+10
BTC,21000000,1.792718e+07
ETH,0,1.076842e+08
...,...,...
ZEPH,2000000000,2.000000e+09
GAP,250000000,1.493105e+07
BDX,1400222610,9.802226e+08
ZEN,21000000,7.296538e+06


In [46]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
cluster_df = clustered_df[['TotalCoinSupply', 'TotalCoinsMined']]
X_minmax = MinMaxScaler().fit_transform(cluster_df)
X_minmax

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [47]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
index_values = (clustered_df.index.tolist())
plot_df = pd.DataFrame(
    data = X_minmax, columns=["TotalCoinSupply_scaled", "TotalCoinsMined_scaled"], index = index_values)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df = plot_df.join(cc_names_df, how='inner')

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
class_df = clustered_df['Class']
plot_df = plot_df.join(class_df, how='inner')

plot_df.head(10)

,TotalCoinSupply_scaled,TotalCoinsMined_scaled,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,2
ETH,0.000000e+00,0.000109,Ethereum,2
LTC,8.400000e-05,0.000064,Litecoin,2
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,2
ETC,2.100000e-04,0.000115,Ethereum Classic,2
ZEC,2.100000e-05,0.000007,ZCash,2


In [49]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# hvplot not working
supply_mined_df.hvplot.scatter(y="TotalCoinsMined", x="TotalCoinSupply")


AttributeError: 'DataFrame' object has no attribute 'hvplot'